In [ ]:
import pandas as pd
import numpy as np

In [ ]:
folder = 'no_protected_model'  # Folder in results containing data and where analysis will be saved

In [ ]:
preds = np.load(f'results/{folder}/pred.npy')
testX = np.load(f'results/{folder}/testX.npy')
testY = np.load(f'results/{folder}/testY.npy')
print(f"Shapes: {preds.shape}, {testX.shape}, {testY.shape}")

In [ ]:
# Import partially preprocessed version so there aren't too many columns
test_df = pd.read_csv('dataset/test_split_partially_preprocessed.csv')
# test_df = pd.read_csv('dataset/test_preprocessed.csv')  # Full dataset

# Add truncation (since this wasn't done for partially_preprocessed version)
test_df.loc[test_df.INCWAGE_CPIU_2010 > 100000, 'INCWAGE_CPIU_2010'] = 100000

test_df.shape

In [ ]:
testY[:10]

In [ ]:
pd.Series(testY).head(10)

In [ ]:
# This ensures that the indices are correct
assert((test_df.INCWAGE_CPIU_2010 == pd.Series(testY)).all())

In [ ]:
# Check if SERIAL or PERNUM are in testX
# for i in range(testX.shape[1]):
#     seriesConverted = pd.Series(testX[:,i])
#     if (test_df.SERIAL == seriesConverted).all():
#         print(f"Column {i} is SERIAL")
#     if (test_df.PERNUM == seriesConverted).all():
#         print(f"Column {i} is PERNUM")

In [ ]:
print("Old shape: ", test_df.shape)
test_df['Income_Pred'] = preds
print("New shape: ", test_df.shape)

In [ ]:
# These are metrics with no rounding
test_df['Income_Pred'].describe()

In [ ]:
test_df['Income_Pred'] = test_df['Income_Pred'].round(0)

In [ ]:
# These are metrics with rounding
test_df['Income_Pred'].describe()

In [ ]:
test_df.INCWAGE_CPIU_2010.describe()

## Look for biases

### Create underprediction/overprediction data

In [ ]:
# Number of predictions that are perfect
(test_df.Income_Pred.round(0) == test_df.INCWAGE_CPIU_2010).values.sum()

In [ ]:
test_df['Pred_Error'] = test_df['Income_Pred'] - test_df['INCWAGE_CPIU_2010']
test_df['Pred_Error'].describe()

In [ ]:
test_df['Pred_AbsError'] = test_df['Pred_Error'].abs()
test_df['Pred_AbsError'].describe()

In [ ]:
print("Income summary")
print(test_df.INCWAGE_CPIU_2010.describe())
print("\nIncome prediction summary")
print(test_df.Income_Pred.describe())
print("\nAbsolute error summary")
print(test_df.Pred_AbsError.describe())
print("\nRelative error summary")
print(test_df.Pred_Error.describe())

In [ ]:
(~full_df.isWhite).sum()

In [ ]:
full_df.isWhite.values.sum()

In [ ]:
test_df.isWhite.values.sum()

In [ ]:
train_df.isWhite.values.sum()

In [ ]:
full_df.isAsian.sum()

In [ ]:
full_df.isWhite.sum() / len(full_df)

### Test differences in accuracy and under/overprediction rates

In [ ]:
test_df.columns

In [ ]:
protected_cols = [
    'isFemale', 
    'isAmericanIndian', 'isAsian', 'isBlack', 'isPacificIslander', 'isWhite', 'isOtherRace', 'isHispanic',
    'bornInUS',
    'isMarried', 'wasMarried', 'neverMarried',
    'sameSexMarriage', 'mixedRaceMarriage',
]

In [ ]:
totalRows = len(test_df)  # Total number of rows in the dataset
summaryEntries = 0  # Number of entries in the summary

with open(f'results/{folder}/analysis/summary1.txt', 'w') as f:
    for col in protected_cols:
        if test_df[col].dtype != 'bool':
            raise Exception(f"Column {col} is not boolean")

        assert(not test_df[col].isna().values.any())

        numTrue = test_df[col].values.sum()  # Number of entries for which the column is true
        numFalse = totalRows - numTrue  # Number of entries for which the column is false

        dfTrue = test_df[test_df[col]]
        dfFalse = test_df[~test_df[col]]

        print(f'Of the people for whom {col} is true ({numTrue} of {totalRows} entries, or {numTrue / totalRows * 100}%), actual salaries are:')
        print(dfTrue.INCWAGE_CPIU_2010.describe())
        print("Predictions are")
        print(dfTrue.Income_Pred.describe())
        print("Absolute error is:")
        print(dfTrue.Pred_AbsError.describe())
        print("Relative error is:")
        print(dfTrue.Pred_Error.describe())
        print("\n")
        print(f'Of the people for whom {col} is false ({numFalse} of {totalRows} entries, or {numFalse / totalRows * 100}%), actual salaries are:')
        print(dfFalse.INCWAGE_CPIU_2010.describe())
        print("Predictions are")
        print(dfFalse.Income_Pred.describe())
        print("Absolute error is:")
        print(dfFalse.Pred_AbsError.describe())
        print("Relative error is:")
        print(dfFalse.Pred_Error.describe())
        print("\n\n")

        # Look for interesting cases to add to summary

        # Large difference in mean prediction
        if(abs(dfTrue.Income_Pred.mean() - dfFalse.Income_Pred.mean()) > 1000):
            f.write(f"Mean prediction for {col} is significantly different than mean prediction for not {col}\n")
            f.write(f"Mean prediction for {col}: {dfTrue.Income_Pred.mean()}")
            f.write(f"\tMean prediction for not {col}: {dfFalse.Income_Pred.mean()}")
            f.write("\n\n")
            summaryEntries += 1
        # Large difference in mean absolute error
        if(abs(dfTrue.Pred_AbsError.mean() - dfFalse.Pred_AbsError.mean()) > 1000):
            f.write(f"Mean absolute error for {col} is significantly different than mean absolute error for not {col}\n")
            f.write(f"Mean absolute error for {col}: {dfTrue.Pred_AbsError.mean()}")
            f.write(f"\tMean absolute error for not {col}: {dfFalse.Pred_AbsError.mean()}")
            f.write("\n\n")
            summaryEntries += 1
        # Large difference in mean relative error
        if(abs(dfTrue.Pred_Error.mean() - dfFalse.Pred_Error.mean()) > 750):
            f.write(f"Mean relative error for {col} is significantly different than mean relative error for not {col}\n")
            f.write(f"Mean relative error for {col}: {dfTrue.Pred_Error.mean()}")
            f.write(f"\tMean relative error for not {col}: {dfFalse.Pred_Error.mean()}")
            f.write("\n\n")
            summaryEntries += 1

In [ ]:
summaryEntries

## Miscellaneous analysis

In [ ]:
test_df.AGE.describe()

In [ ]:
print("Age distribution for people who are married")
print(test_df[test_df.isMarried].AGE.describe())
print("Income distribution for people who are married")
print(test_df[test_df.isMarried].INCWAGE_CPIU_2010.describe())
print("\nAge distribution for people who are not married")
print(test_df[~test_df.isMarried].AGE.describe())
print("Income distribution for people who are not married")
print(test_df[~test_df.isMarried].INCWAGE_CPIU_2010.describe())

In [ ]:
print("Age distribution for people who were married")
print(test_df[test_df.wasMarried].AGE.describe())
print("Income distribution for people who were married")
print(test_df[test_df.wasMarried].INCWAGE_CPIU_2010.describe())
print("\nAge distribution for people who were not married")
print(test_df[~test_df.wasMarried].AGE.describe())
print("Income distribution for people who were not married")
print(test_df[~test_df.wasMarried].INCWAGE_CPIU_2010.describe())

In [ ]:
temp_df = test_df.copy()
age_cols = []

for i in range(1, 10):
    col = f'age_{i*10}To{(i+1)*10}'
    print(f'Of the people for whom {col} is true, income is')
    print(test_df[(test_df.AGE >= i*10) & (test_df.AGE < (i+1)*10)].INCWAGE_CPIU_2010.describe())
    print(f'Of the people for whom {col} is false, income is')
    print(test_df[(test_df.AGE < i*10) | (test_df.AGE >= (i+1)*10)].INCWAGE_CPIU_2010.describe())
    print("\n")

In [ ]:
test_df[test_df.isFemale].INCWAGE_CPIU_2010.describe()

In [ ]:
test_df[~test_df.isFemale].INCWAGE_CPIU_2010.describe()

In [ ]:
[c for c in test_df.columns if 'DEGFIELD' in c]

In [ ]:
train_df = pd.read_csv('dataset/train_split_partially_preprocessed.csv')
train_df.shape

In [ ]:
test_df.shape

In [ ]:
full_df = pd.concat([train_df, test_df])
full_df.shape

In [ ]:
print("Ratio are women:", full_df.isFemale.values.sum() / full_df.shape[0])

In [ ]:
# Number with business degree
print("Women:", (full_df.isFemale & (full_df.DEGFIELD == 62)).values.sum())
print("Men:", (~full_df.isFemale & (full_df.DEGFIELD == 62)).values.sum())
print("Percent are women:", (full_df.isFemale & (full_df.DEGFIELD == 62)).values.sum() / len(full_df))
print("Percent are men:", (~full_df.isFemale & (full_df.DEGFIELD == 62)).values.sum() / len(full_df))
print("Ratio women:", (full_df.isFemale & (full_df.DEGFIELD == 62)).values.sum() / ((~full_df.isFemale & (full_df.DEGFIELD == 62)).values.sum()+(full_df.isFemale & (full_df.DEGFIELD == 62)).values.sum()))

In [ ]:
# Number with engineering degree
print("Women:", (full_df.isFemale & (full_df.DEGFIELD == 24)).values.sum())
print("Men:", (~full_df.isFemale & (full_df.DEGFIELD == 24)).values.sum())
print("Percent are women:", (full_df.isFemale & (full_df.DEGFIELD == 24)).values.sum() / len(full_df))
print("Percent are men:", (~full_df.isFemale & (full_df.DEGFIELD == 24)).values.sum() / len(full_df))
print("Ratio women:", (full_df.isFemale & (full_df.DEGFIELD == 24)).values.sum() / ((~full_df.isFemale & (full_df.DEGFIELD == 24)).values.sum()+(full_df.isFemale & (full_df.DEGFIELD == 24)).values.sum()))

In [ ]:
# Number with some college
print("Women:", (full_df.isFemale & (full_df.someCollege)).values.sum())
print("Men:", (~full_df.isFemale & (full_df.someCollege)).values.sum())
print("Percent are women:", (full_df.isFemale & (full_df.someCollege)).values.sum() / len(full_df))
print("Percent are men:", (~full_df.isFemale & (full_df.someCollege)).values.sum() / len(full_df))
print("Ratio are women:", (full_df.isFemale & (full_df.someCollege)).values.sum() / ((~full_df.isFemale & (full_df.someCollege)).values.sum()+(full_df.isFemale & (full_df.someCollege)).values.sum()))

In [ ]:
# Number working in california
print("Women:", (full_df.isFemale & (full_df.PWSTATE2 == 6)).values.sum())
print("Men:", (~full_df.isFemale & (full_df.PWSTATE2 == 6)).values.sum())
print("Percent are women:", (full_df.isFemale & (full_df.PWSTATE2 == 6)).values.sum() / len(full_df))
print("Percent are men:", (~full_df.isFemale & (full_df.PWSTATE2 == 6)).values.sum() / len(full_df))
print("Ratio are women:", (full_df.isFemale & (full_df.PWSTATE2 == 6)).values.sum() / ((~full_df.isFemale & (full_df.PWSTATE2 == 6)).values.sum()+(full_df.isFemale & (full_df.PWSTATE2 == 6)).values.sum()))

In [ ]:
# Number working as "Chief executives and legislators/public administration"
print("Women:", (full_df.isFemale & (full_df.OCC2010 == 10)).values.sum())
print("Men:", (~full_df.isFemale & (full_df.OCC2010 == 10)).values.sum())
print("Percent are women:", (full_df.isFemale & (full_df.OCC2010 == 10)).values.sum() / len(full_df))
print("Percent are men:", (~full_df.isFemale & (full_df.OCC2010 == 10)).values.sum() / len(full_df))
print("Ratio of women:", (full_df.isFemale & (full_df.OCC2010 == 10)).values.sum() / ((~full_df.isFemale & (full_df.OCC2010 == 10)).values.sum()+(full_df.isFemale & (full_df.OCC2010 == 10)).values.sum()))

In [ ]:
# Number working as "Physicians and Surgeons"
print("Women:", (full_df.isFemale & (full_df.OCC2010 == 3060)).values.sum())
print("Men:", (~full_df.isFemale & (full_df.OCC2010 == 3060)).values.sum())
print("Percent are women:", (full_df.isFemale & (full_df.OCC2010 == 3060)).values.sum() / len(full_df))
print("Percent are men:", (~full_df.isFemale & (full_df.OCC2010 == 3060)).values.sum() / len(full_df))
print("Ratio of women:", (full_df.isFemale & (full_df.OCC2010 == 3060)).values.sum() / ((~full_df.isFemale & (full_df.OCC2010 == 3060)).values.sum()+(full_df.isFemale & (full_df.OCC2010 == 3060)).values.sum()))

In [ ]:
# These are codes and names of jobs that were prioritized by the model in the shap analysis
shapOccs = [
    (430, "Managers, nec (including Postmasters)"),
    (3130, "Registered nurses"),
    (10, "Chief executives and legislators/public administration"),
    (3060, "Physicians and Surgeons"),
    ((4840, 4850), "Sales Representatives")
]

In [ ]:
for occNum, occName in shapOccs:
    print(occNum, type(occNum) is tuple)

In [ ]:
raceCols = ['isWhite', 'isBlack', 'isAsian', 'isAmericanIndian', 'isPacificIslander', 'isOtherRace']

In [ ]:
[c for c in full_df.columns if 'School' in c]

In [ ]:
for occNum, occName in shapOccs:
    print(f"For occupation {occName}:")
    if type(occNum) is tuple:
        assert(len(occNum) == 2), "occNum must be a tuple of length 2"
        occDf = full_df[(full_df.OCC2010 == occNum[0]) | (full_df.OCC2010 == occNum[1])]
    else:
        occDf = full_df[full_df.OCC2010 == occNum]
    print("Number women:", occDf.isFemale.values.sum(), 
        # "\tNumber men:", (~occDf.isFemale).values.sum(), 
        "\tPercent women:", occDf.isFemale.values.sum() / len(occDf)
    )

    for race in raceCols:
        print(f"Number {race}:", occDf[race].values.sum(), f"\tPercent {race}:", occDf[race].values.sum() / len(occDf))
    
    print("\n")

In [ ]:
# These are codes and names of jobs that were prioritized by the model in the shap analysis
shapDegrees = [
    (62, "Business"),
    (24, "Engineering")
]

In [ ]:
for degNum, degName in shapDegrees:
    print(f"For degree {degName}:")
    degDf = full_df[full_df.DEGFIELD == degNum]
    print("Number women:", degDf.isFemale.values.sum(),
        "\tPercent women:", degDf.isFemale.values.sum() / len(degDf)
    )

    for race in raceCols:
        print(f"Number {race}:", degDf[race].values.sum(), f"\tPercent {race}:", degDf[race].values.sum() / len(degDf))
    
    print("\n")

In [ ]:
full_df[full_df.isSelfEmployed].INCWAGE_CPIU_2010.describe()

In [ ]:
full_df[~full_df.isSelfEmployed].INCWAGE_CPIU_2010.describe()

In [ ]:
someCollegeDf = full_df[full_df.someCollege]
notSomeCollegeDf = full_df[~full_df.someCollege]

print("For someCollege=True:")
print("Number women:", someCollegeDf.isFemale.values.sum(),
    "\tPercent women:", someCollegeDf.isFemale.values.sum() / len(someCollegeDf)
)

for race in raceCols:
    print(f"Number {race}:", someCollegeDf[race].values.sum(), 
        f"\tPercent {race}:", someCollegeDf[race].values.sum() / len(someCollegeDf)
    )

print("\nFor someCollege=False:")
print("Number women:", notSomeCollegeDf.isFemale.values.sum(),
    "\tPercent women:", notSomeCollegeDf.isFemale.values.sum() / len(notSomeCollegeDf)
)

for race in raceCols:
    print(f"Number {race}:", notSomeCollegeDf[race].values.sum(), 
        f"\tPercent {race}:", notSomeCollegeDf[race].values.sum() / len(notSomeCollegeDf)
    )

In [ ]:
[c for c in full_df.columns if 'Sales_Representative' in c]

In [ ]:
(full_df.OCC2010 == 4850).sum()

In [ ]:
len(full_df)

In [ ]:
[c for c in full_df.columns if 'English' in c]

In [ ]:
(~full_df.speaksEnglish & ~full_df.speaksOnlyEnglish & ~full_df.speaksEnglishWell).values.sum()

In [ ]:
# Number working as "Registered nurses"
print("Women:", (full_df.isFemale & (full_df.OCC2010 == 3130)).values.sum())
print("Men:", (~full_df.isFemale & (full_df.OCC2010 == 3130)).values.sum())
print("Percent are women:", (full_df.isFemale & (full_df.OCC2010 == 3130)).values.sum() / len(full_df))
print("Percent are men:", (~full_df.isFemale & (full_df.OCC2010 == 3130)).values.sum() / len(full_df))
print("Ratio of women:", (full_df.isFemale & (full_df.OCC2010 == 3130)).values.sum() / ((~full_df.isFemale & (full_df.OCC2010 == 3130)).values.sum()+(full_df.isFemale & (full_df.OCC2010 == 3130)).values.sum()))

In [ ]:
# Number working as "Chief executives and legislators/public administration"
print("Women:", (full_df.isFemale & (full_df.OCC2010 == 10)).values.sum())
print("Men:", (~full_df.isFemale & (full_df.OCC2010 == 10)).values.sum())
print("Percent are women:", (full_df.isFemale & (full_df.OCC2010 == 10)).values.sum() / len(full_df))
print("Percent are men:", (~full_df.isFemale & (full_df.OCC2010 == 10)).values.sum() / len(full_df))
print("Ratio of women:", (full_df.isFemale & (full_df.OCC2010 == 10)).values.sum() / ((~full_df.isFemale & (full_df.OCC2010 == 10)).values.sum()+(full_df.isFemale & (full_df.OCC2010 == 10)).values.sum()))

In [ ]:
(~full_df.isFemale & (full_df.DEGFIELD == 62)).values.sum()

In [ ]:
full_df[full_df.someCollege].INCWAGE_CPIU_2010.describe()

In [ ]:
full_df[~full_df.someCollege].INCWAGE_CPIU_2010.describe()

In [ ]:
totalRows = len(test_df)  # Total number of rows in the dataset
summaryEntries = 0  # Number of entries in the summary

with open(f'results/{folder}/analysis/summary1.txt', 'w') as f:
    for col in protected_cols:
        if test_df[col].dtype != 'bool':
            raise Exception(f"Column {col} is not boolean")

        assert(not test_df[col].isna().values.any())

        numTrue = test_df[col].values.sum()  # Number of entries for which the column is true
        numFalse = totalRows - numTrue  # Number of entries for which the column is false

        dfTrue = test_df[test_df[col]]
        dfFalse = test_df[~test_df[col]]

        print(f'Of the people for whom {col} is true ({numTrue} of {totalRows} entries, or {numTrue / totalRows * 100}%), actual salaries are:')
        print(dfTrue.INCWAGE_CPIU_2010.describe())
        print("Predictions are")
        print(dfTrue.Income_Pred.describe())
        print("Absolute error is:")
        print(dfTrue.Pred_AbsError.describe())
        print("Relative error is:")
        print(dfTrue.Pred_Error.describe())
        print("\n")
        print(f'Of the people for whom {col} is false ({numFalse} of {totalRows} entries, or {numFalse / totalRows * 100}%), actual salaries are:')
        print(dfFalse.INCWAGE_CPIU_2010.describe())
        print("Predictions are")
        print(dfFalse.Income_Pred.describe())
        print("Absolute error is:")
        print(dfFalse.Pred_AbsError.describe())
        print("Relative error is:")
        print(dfFalse.Pred_Error.describe())
        print("\n\n")

        # Look for interesting cases to add to summary

        # Large difference in mean prediction
        if(abs(dfTrue.Income_Pred.mean() - dfFalse.Income_Pred.mean()) > 1000):
            f.write(f"Mean prediction for {col} is significantly different than mean prediction for not {col}\n")
            f.write(f"Mean prediction for {col}: {dfTrue.Income_Pred.mean()}")
            f.write(f"\tMean prediction for not {col}: {dfFalse.Income_Pred.mean()}")
            f.write("\n\n")
            summaryEntries += 1
        # Large difference in mean absolute error
        if(abs(dfTrue.Pred_AbsError.mean() - dfFalse.Pred_AbsError.mean()) > 1000):
            f.write(f"Mean absolute error for {col} is significantly different than mean absolute error for not {col}\n")
            f.write(f"Mean absolute error for {col}: {dfTrue.Pred_AbsError.mean()}")
            f.write(f"\tMean absolute error for not {col}: {dfFalse.Pred_AbsError.mean()}")
            f.write("\n\n")
            summaryEntries += 1
        # Large difference in mean relative error
        if(abs(dfTrue.Pred_Error.mean() - dfFalse.Pred_Error.mean()) > 750):
            f.write(f"Mean relative error for {col} is significantly different than mean relative error for not {col}\n")
            f.write(f"Mean relative error for {col}: {dfTrue.Pred_Error.mean()}")
            f.write(f"\tMean relative error for not {col}: {dfFalse.Pred_Error.mean()}")
            f.write("\n\n")
            summaryEntries += 1